In [24]:
# example of training a final classification model
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets.samples_generator import make_blobs

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import pickle

import numpy as np
import pandas as pd

In [25]:
path = '../Entrenamientos/Procesado_Tramos.xlsx'
df = pd.read_excel(path)
df = df[df["porc"]!=np.inf]
df["puntos_en_tramo"]= df["tramo_final"]-df["tramo_inicial"]
df = df[df["puntos_en_tramo"]>9]

df2 = df[["pwr","porc","viento_aparente","spd"]]

dataset = df2.values

In [26]:
# generate 2d classification dataset
X = dataset[:,0:3]
Y = dataset[:,3]

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [28]:
#standarScaler

#scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
#scalarX, scalarY = StandardScaler(), StandardScaler()
scalarX, scalarY = RobustScaler(), RobustScaler()

scalarX.fit(X)
#scalarY.fit(Y.reshape(len(Y),1))
X = scalarX.transform(X)

In [29]:
#Probar por diferentes funciones de activacion y con regularizacion

# define and fit the final model
model = Sequential()
model.add(Dense(4, input_dim=3, activation='relu'))

model.add(Dense(60, activation='relu'))

model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(X, Y, epochs=1000, verbose=0)

In [30]:
Xnew = scalarX.transform(X_test)


# make a prediction
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
        print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))


X=[-0.562863   -1.12386178  0.37752359], Predicted=[46.471947]
X=[0.         0.         0.49346415], Predicted=[34.873512]
X=[0.80419191 1.21801837 0.57660498], Predicted=[20.8841]
X=[-0.18215847 -0.051804    1.102153  ], Predicted=[39.495636]
X=[ 0.17867177 -0.60855099  0.50892582], Predicted=[42.78855]
X=[-1.18035653 -1.44710551 -0.00456275], Predicted=[45.111027]
X=[-0.09493155 -0.50624808 -0.80067917], Predicted=[32.55168]
X=[-0.8456521  -0.79827822 -0.82372021], Predicted=[32.50351]
X=[-0.27556754 -0.10080363 -0.21070516], Predicted=[30.523853]
X=[-0.11303955  0.33553766  0.3947009 ], Predicted=[28.91097]
X=[-0.97219242 -0.27426341  0.23580307], Predicted=[30.998625]
X=[-0.01574595  0.49864022 -0.45447419], Predicted=[23.828228]
X=[0.03005714 0.11713011 0.15879575], Predicted=[31.468616]
X=[-1.40043817 -0.24153485 -0.02997483], Predicted=[28.038628]
X=[0.62185907 0.79612186 0.48835572], Predicted=[27.452492]
X=[ 1.00154241  1.07293192 -0.31870205], Predicted=[24.859983]
X=[-0.0740

In [31]:
test_val = pd.DataFrame(X_test)
test_val.columns = (["pwr","porc","viento_aparente"])
test_val["spd"] = Y_test
test_val["y_pred"] = ynew
test_val["diff"] = test_val["spd"]-test_val["y_pred"]
test_val.head()

,pwr,porc,viento_aparente,spd,y_pred,diff
0,123.212955,-3.276340,9.441216,46.958144,46.471947,0.486197
1,175.647415,-0.176732,12.344353,29.340000,34.873512,-5.533512
2,250.563280,3.182560,14.426188,22.531272,20.884100,1.647172
3,158.678133,-0.319607,27.585842,42.441487,39.495636,2.945851
4,192.291888,-1.855114,12.731511,44.639300,42.788551,1.850748


In [32]:
print("Predicciones:")
print(test_val["y_pred"].mean())
print(test_val["y_pred"].max())
print(test_val["y_pred"].min())

print("Diferencias:")
print(test_val["diff"].mean())
print(test_val["diff"].max())
print(test_val["diff"].min())

Predicciones:
31.63483238220215
56.188583
17.293713
Diferencias:
0.18003000268594907
14.514907925828645
-28.881593632172287


In [33]:
test_val[(test_val["diff"]>8) | (test_val["diff"]<-10)]

,pwr,porc,viento_aparente,spd,y_pred,diff
17,43.281740,-4.173012,14.445208,61.576779,47.061871,14.514908
32,245.479461,1.437892,14.238703,40.524726,31.647793,8.876934
36,122.686452,-0.670212,14.831182,2.375887,31.257481,-28.881594
59,220.011123,-1.406858,4.903130,24.042467,39.378113,-15.335646
75,29.602529,-6.597866,14.398382,69.830249,56.188583,13.641665
86,52.584995,-2.298785,5.659875,20.482496,36.006115,-15.523619
95,157.716371,0.371855,-10.881710,35.510085,25.744022,9.766062
102,152.194473,-3.029458,-4.324675,22.892224,41.984295,-19.092071
105,223.945818,1.835691,20.569671,18.824834,29.344618,-10.519784
110,38.010882,-1.649214,11.187904,14.647389,27.647043,-12.999654


In [34]:
len(test_val)

232

In [35]:
pickle.dump(scalarX, open("Scaler.model", 'wb'))


model.save("TF_Tramos_2.model")
#Para cargar el modelo
#https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model